In [ ]:
# Importa as bibliotecas necessárias para a conexão com o banco de dados ODBC

import pyodbc
import pandas as pd
import configparser
import warnings
from datetime import datetime

# Filtra o aviso procurando pelo texto exato da mensagem
warnings.filterwarnings(
    'ignore',
    message='pandas only supports SQLAlchemy.*' # Usamos a mensagem como filtro
)

print("Bibliotecas importadas e aviso configurado com sucesso!")

Bibliotecas importadas e aviso configurado com sucesso!


In [ ]:
# Configura a função para solicitar e validar as datas de início e fim da consulta, além dos nomes dos arquivos de entrada e saída

def solicitar_data(prompt):
    while True:
        data_str = input(prompt)
        try:
            # Tenta converter a string para data no formato DD/MM/AAAA
            data_obj = datetime.strptime(data_str, '%d/%m/%Y')
            return data_obj
        except ValueError:
            print("❌ Formato de data inválido. Por favor, use o formato DD/MM/AAAA.")

print("Por favor, informe o período para a consulta de faturamento.")
data_inicial = solicitar_data("Digite a data inicial (DD/MM/AAAA): ")
data_final = solicitar_data("Digite a data final (DD/MM/AAAA):   ")

# Validação para garantir que a data final não seja anterior à inicial
while data_final < data_inicial:
    print("❌ A data final não pode ser anterior à data inicial. Por favor, insira as datas novamente.")
    data_inicial = solicitar_data("Digite a data inicial (DD/MM/AAAA): ")
    data_final = solicitar_data("Digite a data final (DD/MM/AAAA):   ")


# Formatando as datas para o formato que o arquivo SQL espera (AAAA-MM-DD)
DATA_INICIAL_SQL = data_inicial.strftime('%Y-%m-%d')
DATA_FINAL_SQL = data_final.strftime('%Y-%m-%d')

# Nomes dos arquivos de entrada e saída
ARQUIVO_CNPJS = 'cnpjs.txt'
ARQUIVO_SQL = 'faturamento.sql'
ARQUIVO_SAIDA_CSV = 'faturamento_final.csv'

print("\n✅ Datas definidas com sucesso!")
print(f"🗓️ Período de apuração: de {DATA_INICIAL_SQL} a {DATA_FINAL_SQL}")

Por favor, informe o período para a consulta de faturamento.


Digite a data inicial (DD/MM/AAAA):  01012024


❌ Formato de data inválido. Por favor, use o formato DD/MM/AAAA.


Digite a data inicial (DD/MM/AAAA):  01/01/2024
Digite a data final (DD/MM/AAAA):    31/12/2024



✅ Datas definidas com sucesso!
🗓️ Período de apuração: de 2024-01-01 a 2024-12-31


In [ ]:
# Configura e estabelece a conexão com o banco de dados ODBC

# Ler as configurações do arquivo config.ini
config = configparser.ConfigParser()
config.read('config.ini')

# Pegar as credenciais da seção [DATABASE]
dsn_name = config['DATABASE']['DSN']
user = config['DATABASE']['USER']
password = config['DATABASE']['PASSWORD']

# Montando a string de conexão
connection_string = f'DSN={dsn_name};UID={user};PWD={password}'

# Bloco try...except para tratar possíveis erros de conexão
try:
    # Tenta conectar ao banco de dados usando pyodbc
    conn = pyodbc.connect(connection_string, autocommit=True)
    
    # Se chegou até aqui, a conexão foi um sucesso!
    print("✅ Conexão com o banco de dados via pyodbc estabelecida com sucesso!")

except pyodbc.Error as ex:
    sqlstate = ex.args[0]
    print(f"❌ Erro na conexão com o banco de dados.")
    print(f"   Verifique se o nome do DSN ('{dsn_name}') e as credenciais estão corretos.")
    print(f"   Detalhe do erro: {ex}")
    conn = None # Garante que a variável conn não será usada se der erro

✅ Conexão com o banco de dados via pyodbc estabelecida com sucesso!


In [ ]:
# Carrega a query SQL e a lista de CNPJs

query_faturamento_base = ""
lista_cnpjs = []

if conn:
    try:
        with open(ARQUIVO_SQL, 'r', encoding='utf-8') as f:
            query_faturamento_base = f.read()
        print(f"✅ Arquivo SQL '{ARQUIVO_SQL}' carregado.")

        with open(ARQUIVO_CNPJS, 'r', encoding='utf-8') as f:
            # Remove espaços em branco e linhas vazias
            lista_cnpjs = [line.strip() for line in f if line.strip()]
        print(f"✅ Arquivo de CNPJs '{ARQUIVO_CNPJS}' carregado com {len(lista_cnpjs)} CNPJs.")

    except FileNotFoundError as e:
        print(f"❌ Erro: Arquivo não encontrado - {e}. Verifique se os nomes estão corretos.")
        conn = None # Impede a execução da próxima célula

✅ Arquivo SQL 'faturamento.sql' carregado.
✅ Arquivo de CNPJs 'cnpjs.txt' carregado com 160 CNPJs.


In [20]:
# Consulta o faturamento

resultados = []

if conn and lista_cnpjs:
    print("\n--- Iniciando processamento do faturamento ---")
    total_cnpjs = len(lista_cnpjs)

    for i, cnpj in enumerate(lista_cnpjs):
        print(f"\n({i+1}/{total_cnpjs}) Processando CNPJ: {cnpj}")
        
        try:
            # 1. Encontrar o CODI_EMP e a RAZÃO SOCIAL a partir do CNPJ
            # A coluna CGCE_EMP geralmente armazena CNPJ/CPF e a NOME_EMP a razão social.
            query_codi_emp = f"SELECT codi_emp, nome_emp FROM bethadba.geempre WHERE cgce_emp = '{cnpj}'"
            df_empresa = pd.read_sql(query_codi_emp, conn)

            if df_empresa.empty:
                print(f"   ⚠️  Aviso: Nenhuma empresa encontrada para o CNPJ {cnpj}.")
                resultados.append({'CNPJ': cnpj, 'RAZAO_SOCIAL': 'NÃO ENCONTRADA', 'FATURAMENTO_TOTAL': 'NÃO ENCONTRADO'})
                continue

            codi_emp = df_empresa['codi_emp'].iloc[0]
            razao_social = df_empresa['nome_emp'].iloc[0]
            print(f"   > Empresa encontrada: {razao_social} (Cód: {codi_emp})")

            # 2. Preparar e executar a consulta de faturamento
            # Substituímos os parâmetros na sua query original
            query_modificada = query_faturamento_base.replace(':ALLISTAEMPRESAS', str(codi_emp))
            query_modificada = query_modificada.replace(':DATINI', f"'{DATA_INICIAL_SQL}'")
            query_modificada = query_modificada.replace(':DATFIN', f"'{DATA_FINAL_SQL}'")
            # Adicione outras substituições se houver mais parâmetros
            query_modificada = query_modificada.replace(":CIDAEMP", "'CIDADE'")
            query_modificada = query_modificada.replace(":CEPEEMP", "'CEP'")
            query_modificada = query_modificada.replace(":ENDEEMP", "'ENDERECO'")
            query_modificada = query_modificada.replace(":EXLUIR_CUSTO_AQUISICAO_VEICULO", "'N'")


            # 3. Agregamos a query complexa para obter um único valor de faturamento
            query_final_agregada = f"""
                SELECT
                    SUM(COALESCE(VSAI, 0) + COALESCE(VSER, 0) + COALESCE(VOUT, 0)) as FATURAMENTO_TOTAL
                FROM (
                    {query_modificada}
                ) AS subquery
            """

            df_faturamento = pd.read_sql(query_final_agregada, conn)

            if not df_faturamento.empty:
                faturamento_total = df_faturamento['FATURAMENTO_TOTAL'].iloc[0]
                faturamento_total = faturamento_total if pd.notna(faturamento_total) else 0
                print(f"   > Faturamento encontrado: R$ {faturamento_total:,.2f}")
                resultados.append({'CNPJ': cnpj, 'RAZAO_SOCIAL': razao_social, 'FATURAMENTO_TOTAL': faturamento_total})
            else:
                print("   > Nenhum dado de faturamento retornado para o período.")
                resultados.append({'CNPJ': cnpj, 'RAZAO_SOCIAL': razao_social, 'FATURAMENTO_TOTAL': 0.0})

        except Exception as e:
            print(f"   ❌ Erro ao processar o CNPJ {cnpj}: {e}")
            # Tenta adicionar a razão social mesmo em caso de erro, se já tiver sido encontrada
            razao_social_erro = razao_social if 'razao_social' in locals() else 'ERRO NA BUSCA'
            resultados.append({'CNPJ': cnpj, 'RAZAO_SOCIAL': razao_social_erro, 'FATURAMENTO_TOTAL': 'ERRO'})

    print("\n--- Processamento concluído ---")


--- Iniciando processamento do faturamento ---

(1/160) Processando CNPJ: 52176809000172
   > Empresa encontrada: 52.176.809 DOMINGOS JOSE MARIANO DOS SAN (Cód: 578)
   > Faturamento encontrado: R$ 60,100.00

(2/160) Processando CNPJ: 08898489000172
   > Empresa encontrada: A P SANTOS (Cód: 571)
   > Faturamento encontrado: R$ 390,224.68

(3/160) Processando CNPJ: 00317929000149
   > Empresa encontrada: ABAETE LINHAS AEREAS LTDA (Cód: 332)
   > Faturamento encontrado: R$ 0.00

(4/160) Processando CNPJ: 02758960000140
   > Empresa encontrada: ACCIOLY ARAUJO E CIA LTDA. (Cód: 562)
   > Faturamento encontrado: R$ 0.00

(5/160) Processando CNPJ: 13243260000130
   > Empresa encontrada: ADVOCACIA CAIO DRUSO (Cód: 224)
   > Faturamento encontrado: R$ 3,636,680.54

(6/160) Processando CNPJ: 28264218000172
   > Empresa encontrada: AGI CONSTRUÇÕES E LOCAÇOES LTDA (Cód: 277)
   > Faturamento encontrado: R$ 1,244,300.95

(7/160) Processando CNPJ: 02085636000109
   > Empresa encontrada: AGUIAR SER

In [ ]:
# Gera o arquivo de saída ao transformar a lista de resultados num DataFrame e salvá-lo como CSV

if resultados:
    print("\n--- Gerando arquivo de resultado ---")
    df_resultados = pd.DataFrame(resultados)
    
    def formatar_moeda_br(valor):
        """Formata um número para o padrão de moeda brasileiro (ex: 1.234,56)."""
        if not isinstance(valor, (int, float)):
            return valor # Retorna strings como 'ERRO' ou 'NÃO ENCONTRADO' como estão.
        
        # Formata o número (ex: 1234.56 -> "1,234.56")
        valor_formatado_en = f"{valor:,.2f}"
        # Converte para o padrão brasileiro (ex: "1,234.56" -> "1.234,56")
        valor_formatado_br = valor_formatado_en.replace(",", "X").replace(".", ",").replace("X", ".")
        return valor_formatado_br

    # Formatação para melhor visualização no CSV
    if 'FATURAMENTO_TOTAL' in df_resultados.columns:
        # Converte a coluna para tipo numérico, tratando erros (textos viram NaN)
        df_resultados['FATURAMENTO_TOTAL_NUM'] = pd.to_numeric(df_resultados['FATURAMENTO_TOTAL'], errors='coerce')
        
        # Aplica a função de formatação na coluna numérica
        df_resultados['FATURAMENTO_TOTAL_FORMATADO'] = df_resultados['FATURAMENTO_TOTAL_NUM'].apply(formatar_moeda_br)
        
        # Onde a formatação resultou em 'None' (para os erros), preenche com o valor original.
        df_resultados['FATURAMENTO_TOTAL_FORMATADO'] = df_resultados['FATURAMENTO_TOTAL_FORMATADO'].fillna(df_resultados['FATURAMENTO_TOTAL'])

    try:
        # Seleciona as colunas finais para o CSV, na ordem desejada
        df_final_para_csv = df_resultados[['CNPJ', 'RAZAO_SOCIAL', 'FATURAMENTO_TOTAL_FORMATADO']]

        df_final_para_csv.to_csv(
            ARQUIVO_SAIDA_CSV, 
            index=False, 
            sep=';', # Usando ';' para abrir corretamente no Excel brasileiro
            encoding='utf-8-sig',
            header=['CNPJ', 'RAZAO_SOCIAL', 'FATURAMENTO_TOTAL'] # Renomeando as colunas no arquivo final
        )
        print(f"✅ Resultados salvos com sucesso em '{ARQUIVO_SAIDA_CSV}'!")

        print("\nPré-visualização dos resultados:")
        try:
            display(df_final_para_csv)
        except NameError:
            print(df_final_para_csv.to_string())

    except Exception as e:
        print(f"❌ Erro ao salvar o arquivo CSV: {e}")

# Fechar a conexão ao final de tudo
if conn:
    conn.close()
    print("\nConexão com o banco de dados fechada.")


--- Gerando arquivo de resultado ---
✅ Resultados salvos com sucesso em 'faturamento_final.csv'!

Pré-visualização dos resultados:


,CNPJ,RAZAO_SOCIAL,FATURAMENTO_TOTAL_FORMATADO
0,52176809000172,52.176.809 DOMINGOS JOSE MARIANO DOS SAN,"60.100,00"
1,08898489000172,A P SANTOS,"390.224,68"
2,00317929000149,ABAETE LINHAS AEREAS LTDA,"0,00"
3,02758960000140,ACCIOLY ARAUJO E CIA LTDA.,"0,00"
4,13243260000130,ADVOCACIA CAIO DRUSO,"3.636.680,54"
...,...,...,...
155,57558833000190,VALE BENEFICIOS SERVICOS LTDA,"0,00"
156,34279927000124,VAPEÇAS COMÉRCIO DE PEÇAS LTDA-ME,"689.134,26"
157,28746285000123,VIANA E CALDAS ADVOCACIA,"127.748,52"
158,13993421000102,VILLE DESIGN INDUSTRIA COMERCIO E SERVIC,"943.051,18"



Conexão com o banco de dados fechada.
